In [14]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

In [19]:
import keras
from keras.layers import Dense, Conv2D, Flatten
from keras.models import Sequential
from keras.utils import to_categorical

In [7]:
data = pd.read_csv('./pokemon_aug.csv')
data = data.drop(['Total', 'HP', 'Attack', 'Defense','Sp. Atk','Sp. Def',
                  'Speed', 'Generation', 'Legendary'], axis='columns')

In [51]:
mask = np.random.rand(len(data)) < 0.8

In [52]:
train = data[mask].reset_index()
test = data[~mask].reset_index()

In [53]:
gen = ImageDataGenerator(vertical_flip=True, validation_split=0.2)

In [55]:
traingen = gen.flow_from_dataframe(train, directory='./images/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='training')
validgen = gen.flow_from_dataframe(train, directory='./images/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='validation')

Found 250 images belonging to 18 classes.
Found 64 images belonging to 18 classes.


In [56]:
testdatagen = ImageDataGenerator(vertical_flip=False)
testgen = testdatagen.flow_from_dataframe(test, directory='./images/', x_col='fileid', y_col='Type1')

Found 91 images belonging to 18 classes.


In [61]:
model = Sequential()
model.add(Conv2D(128, kernel_size=2, padding='same', activation='relu', 
                 input_shape=(256, 256, 3)))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(Conv2D(32, kernel_size=2, activation='relu'))
model.add(Flatten())
model.add(Dense(18, activation='softmax'))

In [62]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [59]:
STEP_SIZE_TRAIN=traingen.n//traingen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
step_size_test=testgen.n//testgen.batch_size

In [64]:
model.fit_generator(generator=traingen,
                    epochs=3,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

Epoch 1/3
7/7 [==============================] - 67s 10s/step - loss: 14.1105 - val_loss: 13.0960
Epoch 2/3
7/7 [==============================] - 67s 10s/step - loss: 14.8906 - val_loss: 12.5923
Epoch 3/3
7/7 [==============================] - 68s 10s/step - loss: 14.2404 - val_loss: 14.1033


In [ ]:
data.head()

In [ ]:
data.info()